In [ ]:
from langchain_community.document_loaders import TextLoader, pdf
loader = TextLoader('speech.txt')
doc = loader.load()
doc 

In [ ]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
doc=loader.load()

In [ ]:
# we can use pdf and also web pages to read data source from

In [ ]:
#chunks forming form the data
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc = text_splitter.split_documents(doc)
doc

In [ ]:
#vector embeddings and storing the vectr store(chroma, fiass)-> database
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma, FAISS

db_chroma = Chroma.from_documents(doc, OllamaEmbeddings())
db_fiass = FAISS.from_documents(doc, OllamaEmbeddings())

In [ ]:
db_chroma

In [ ]:
db_fiass

In [ ]:
query = 'explain me the exact contecnt in the database'
result1 = db_chroma.similarity_search(query)
result1

In [ ]:
query = 'summarise it'
result2 = db_fiass.similarity_search(query)
result2

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model='llama2')
llm

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
    Answer the following questions based only on the provided context.
    think step by step before providing a detailed answer.
    I will tip you $1000 if the ser finds the answer helpful.
    <context> {context} </context>
    Question: {input}
""")

In [ ]:
#chain 
from langchain.chains.combine_documents import create_stuff_documents_chain

doc_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
retriever=db_fiass.as_retriever()
retriever

In [ ]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,doc_chain)

In [ ]:
response=retrieval_chain.invoke({"input":"Scaled Dot-Product Attention"})